In [1]:
import pandas as pd

In [7]:
def process_ticket_history():
    import pandas as pd

    fans_ticket_history = pd.read_excel('fans_ticket_history_counter.xlsx')
    members_24_25 = pd.read_csv('List_Report_20250625085141.csv')
    members_25_26 = pd.read_csv('List_Report_20250625085424.csv')

    # --- Process 24_25 ---
    members_24_25 = members_24_25[members_24_25['Status'] == 'Active']
    members_24_25['tickets_24_25'] = 1
    members_24_25 = members_24_25[['User Id', 'tickets_24_25']]

    # --- Process 25_26 ---
    members_25_26 = members_25_26[members_25_26['Status'] == 'Active']
    members_25_26['tickets_25_26'] = 1
    members_25_26 = members_25_26[['User Id', 'tickets_25_26']]

    print('fans_ticket_history dtype:', fans_ticket_history['client_number'].dtype)
    print('members_24_25 dtype:', members_24_25['User Id'].dtype)
    print('members_25_26 dtype:', members_25_26['User Id'].dtype)
    print('Shape before processing:', fans_ticket_history.shape)

    # Replace values >1 with 1 (use clip for cleaner solution)
    columns_to_check = [
        'tickets_23_24', 'tickets_22_23', 'tickets_21_22',
        'tickets_19_20', 'tickets_18_19', 'tickets_17_18',
        'tickets_16_17', 'tickets_15_16', 'tickets_14_15'
    ]
    fans_ticket_history[columns_to_check] = fans_ticket_history[columns_to_check].clip(upper=1)

    # Merge members info
    merged = fans_ticket_history.merge(
        members_24_25, left_on='client_number', right_on='User Id', how='left'
    ).merge(
        members_25_26, left_on='client_number', right_on='User Id', how='left'
    )

    # Collect unmatched members (not in ticket history)
    all_members = pd.concat([
        members_24_25[['User Id']],
        members_25_26[['User Id']]
    ]).drop_duplicates()

    unmatched = all_members[~all_members['User Id'].isin(fans_ticket_history['client_number'])].copy()
    unmatched['client_number'] = unmatched['User Id']
    unmatched['tickets_24_25'] = unmatched['User Id'].map(
    members_24_25.drop_duplicates('User Id').set_index('User Id')['tickets_24_25']
    ).fillna(0).astype(int)

    unmatched['tickets_25_26'] = unmatched['User Id'].map(
        members_25_26.drop_duplicates('User Id').set_index('User Id')['tickets_25_26']
    ).fillna(0).astype(int)

    for col in columns_to_check:
        unmatched[col] = 0

    unmatched = unmatched.drop(columns=['User Id'])

    final_df = pd.concat([merged, unmatched], ignore_index=True)

    # Handle client_number and drop extra columns
    if 'User Id_x' in final_df.columns or 'User Id_y' in final_df.columns:
        final_df['client_number'] = final_df['client_number'].fillna(final_df.get('User Id_x')).fillna(final_df.get('User Id_y'))

    final_df = final_df.drop(columns=[col for col in final_df.columns if col.startswith('User Id')])

    # Final cleanup
    columns_to_fill = ['tickets_25_26', 'tickets_24_25'] + columns_to_check
    final_df[columns_to_fill] = final_df[columns_to_fill].fillna(0).astype(int)
    final_df['client_number'] = final_df['client_number'].astype(int)

    # Reorder and export
    final_df = final_df[['client_number'] + columns_to_fill]
    final_df.to_csv('fans_ticket_history_counter_2025_26.csv', index=False)

    return final_df

In [8]:
final_df = process_ticket_history()

fans_ticket_history dtype: int64
members_24_25 dtype: int64
members_25_26 dtype: int64
Shape before processing: (9763, 10)


In [9]:
final_df

,client_number,tickets_25_26,tickets_24_25,tickets_23_24,tickets_22_23,tickets_21_22,tickets_19_20,tickets_18_19,tickets_17_18,tickets_16_17,tickets_15_16,tickets_14_15
0,10003,0,1,1,0,0,0,1,1,0,0,1
1,10010,1,1,1,1,1,1,1,1,1,1,1
2,10011,0,1,1,1,1,1,1,1,1,1,1
3,10011,0,1,1,1,1,1,1,1,1,1,1
4,10011,0,1,1,1,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...
12167,1370948,1,0,0,0,0,0,0,0,0,0,0
12168,1427961,1,0,0,0,0,0,0,0,0,0,0
12169,1427982,1,0,0,0,0,0,0,0,0,0,0
12170,1424290,1,0,0,0,0,0,0,0,0,0,0
